# Fitting a simple model

We have created a dataset in the previous notebook, now it is time to use some data to fit a model.

In [1]:
from mempyfit import *
ds = Dataset()
ds.add(
    name = 't-OD', 
    value = np.array([
        [ -0.05911330049261121,0.10723981900452495],
        [2.0886699507389164,0.10723981900452495],
        [3.9211822660098505,0.12760180995475118],
        [6.029556650246304,0.15656108597285068],
        [8.019704433497536,0.1773755656108597],
        [10.049261083743838,0.19140271493212674],
        [12.039408866995073,0.2],
        [14.088669950738913,0.20769230769230773],
        [16.039408866995075,0.21176470588235297],
        [18.00985221674877,0.21312217194570138]
    ]), 
    units = ['d', '-'], 
    labels = ['time', 'optical density'],
    # optional: add bibkey from Zotero to document literature sources
    bibkey = 'voylesDiversityGrowthPatterns2017' 
)
ds

C:\Users\simon\Documents\mempyfit\mempyfit\dataset.py:62: UserWarning: No temperature given for t-OD and no temperature found in labels.
  warnings.warn(f"No temperature given for {name} and no temperature found in labels.")


<Dataset with 1 entries>
  1. t-OD (UNIVARIATE) [['d', '-']] @ nan K

In [2]:
import matplotlib.pyplot as plt

We always need the same four ingredients to fit a model:

1. A dataset
2. An initial set of parameters
3. A simulator
4. A calibration algorithm


We can collect all of these ingredients in a `ModelFit` object, that will do some of the nitty gritty work for us in the background.

In [9]:
@dataclass
class Parameters:
    names: list[str]
    values: list[float]
    free: list[bool]
    units: list[str]
    labels: list[str]

    def __init__(self, p: dict):

        self.names = []
        self.values = []
        self.free = []
        self.units = []
        self.labels = []

        for (name,info) in p.items():
            self.names.append(name)
            self.values.append(info['value'])
            self.free.append(info['free'])
            self.units.append(info['unit'])
            self.labels.append(info['label'])

        

In [10]:
fit = ModelFit()
fit.data = ds # data
fit.defaultparams = Parameters({
    'r' : {'value' : 0.2, 'free' : True, 'unit' : 'd^-1', 'label' : 'growth rate'}, 
    'K' : {'value' : 1.0, 'free' : True, 'unit' : '-', 'label' : 'carrying capacity'},
})

In [11]:
fit.defaultparams

Parameters(names=['r', 'K'], values=[0.2, 1.0], free=[True, True], units=['d^-1', '-'], labels=['growth rate', 'carrying capacity'])